1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [ ]:
from tensorflow.keras.layers import (
    Flatten, 
    Dense, 
    Dropout, 
    concatenate, 
    Conv2D, 
    Input, 
    BatchNormalization, 
    ReLU, 
    AveragePooling2D, 
    MaxPool2D, 
    GlobalAveragePooling2D
)

from tensorflow.keras.callbacks import (
    EarlyStopping, 
    TensorBoard, 
    ModelCheckpoint, 
    Callback
)

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot as plt

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import datetime
import os

Data

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

170498071/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [ ]:
image_shape = X_train.shape[1:]
print(image_shape)

(32, 32, 3)


In [ ]:
num_classes = len(np.unique(y_train[:, 0]))
print(num_classes)

10


In [ ]:
y_train = to_categorical(y=y_train, num_classes=num_classes)
y_test = to_categorical(y=y_test, num_classes=num_classes)
print(y_train.shape, y_test.shape)

(50000, 10) (10000, 10)


Image generator

Reference: https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

In [ ]:
X_train_image_generator = ImageDataGenerator(rotation_range=15,
                                             horizontal_flip=True,
                                             width_shift_range=0.1,
                                             height_shift_range=0.1,
                                             rescale=1./255).flow(x=X_train, y=y_train)

In [ ]:
X_test_image_generator = ImageDataGenerator(rotation_range=15,
                                            horizontal_flip=True,
                                            width_shift_range=0.1,
                                            height_shift_range=0.1,
                                            rescale=1./255).flow(x=X_test, y=y_test)

DenseNet

Video: https://youtu.be/QKtoh9FJIWQ

Code: https://github.com/Machine-Learning-Tokyo/DL-workshop-series/blob/master/Part%20I%20-%20Convolution%20Operations/ConvNets.ipynb

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def dense_net(image_shape, num_classes, filters=12):
    """
    DenseNet
    """

    def bn_rl_conv(x, f, k=1, s=1, p='same'):
        """
        Dense and Transition block helper.
        """
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters=f, kernel_size=k, strides=s, padding=p)(x)
        return x
    
    def dense_block(tensor, repetitions=36):
        """
        Dense block.
        """
        for _ in range(repetitions):
            x = bn_rl_conv(x=tensor, f=filters//2, k=3)
            tensor = concatenate(inputs=[tensor, x])
        return tensor
    
    def transition_block(x):
        """
        Transition block.
        """
        x = bn_rl_conv(x=x, f=filters//2, k=2)
        x = AveragePooling2D(strides=2, padding='same')(x)
        return x
    
    def output(x):
        """
        Output
        """
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = AveragePooling2D(strides=2, padding='same')(x)
        x = Flatten()(x)
        x = Dense(units=num_classes, activation='softmax')(x)
        return x
    
    input_layer = Input(shape=image_shape)
    x = Conv2D(filters=filters, kernel_size=3, use_bias=False, padding='same')(input_layer)
    
    d1 = dense_block(tensor=x)
    t1 = transition_block(x=d1)

    d2 = dense_block(tensor=t1)
    t2 = transition_block(x=d2)

    d3 = dense_block(tensor=t2)
    t3 = transition_block(x=d3)

    d4 = dense_block(tensor=t3)

    output_layer = output(x=d4)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
model = dense_net(image_shape=image_shape, num_classes=num_classes, filters=12)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 12)   324         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 12)  48          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 re_lu (ReLU)                   (None, 32, 32, 12)   0           ['batch_normalization[0][0]']

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
filepath = "model_save/best_model.h5"
model_save_callback = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# early_stop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.05, patience=10, verbose=1)

log_dir = os.path.join('logs', 'fits', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks = [model_save_callback, tensorboard_callback]

In [ ]:
model.fit(x=X_train_image_generator,
          batch_size=128,
          epochs=300,
          verbose=1,
          validation_data=X_test_image_generator,
          callbacks=callbacks)

Epoch 1/300
1563/1563 [==============================] - ETA: 0s - loss: 1.6477 - accuracy: 0.3936
Epoch 1: val_accuracy improved from -inf to 0.46000, saving model to model_save/best_model.h5
1563/1563 [==============================] - 166s 89ms/step - loss: 1.6477 - accuracy: 0.3936 - val_loss: 1.5638 - val_accuracy: 0.4600
Epoch 2/300
1563/1563 [==============================] - ETA: 0s - loss: 1.2639 - accuracy: 0.5406
Epoch 2: val_accuracy improved from 0.46000 to 0.56380, saving model to model_save/best_model.h5
1563/1563 [==============================] - 133s 85ms/step - loss: 1.2639 - accuracy: 0.5406 - val_loss: 1.2253 - val_accuracy: 0.5638
Epoch 3/300
1563/1563 [==============================] - ETA: 0s - loss: 1.0745 - accuracy: 0.6153
Epoch 3: val_accuracy improved from 0.56380 to 0.62070, saving model to model_save/best_model.h5
1563/1563 [==============================] - 135s 86ms/step - loss: 1.0745 - accuracy: 0.6153 - val_loss: 1.0596 - val_accuracy: 0.6207
Epoch 4

In [17]:
model.load_weights(filepath=filepath)

In [18]:
test_loss, test_accuracy = model.evaluate(x=X_test_image_generator, verbose=1, batch_size=128)

313/313 [==============================] - 8s 25ms/step - loss: 0.4084 - accuracy: 0.8977


In [19]:
print("Test Loss: {}.".format(test_loss))
print("Test Accuracy: {}.".format(test_accuracy * 100))

Test Loss: 0.40838003158569336.
Test Accuracy: 89.77000117301941.


### Please note, I spent more than 7000 INR to purchase compute resources, because I made a mistake in the image augmentation. The mistake was, I added more augmentations which eventually led to the poor performance of the model. I changed the filter values and retrained the model multiple times, which cost me. I am not worried about that.

### I realized lately that I need to reduce image augmentations in order to achieve the desired accuracy.

### Lesson: Just because image augmentations are cool, doesn't mean all augmentations need to be used in the problem.

### If you please notice, the best `val_accuracy` is about 90.56 and the `test_accuracy` is 89.77 which I think is acceptable.

### Please accept my efforts.